In [1]:
import time
import uuid
import os
import cv2
import numpy as np
import mediapipe as mp

objc[2790]: Class CaptureDelegate is implemented in both /Users/odner/opt/miniconda3/envs/torch/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x144d0e538) and /Users/odner/opt/miniconda3/envs/torch/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x1273ac860). One of the two will be used. Which one is undefined.
objc[2790]: Class CVWindow is implemented in both /Users/odner/opt/miniconda3/envs/torch/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x144d0e588) and /Users/odner/opt/miniconda3/envs/torch/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x116110a68). One of the two will be used. Which one is undefined.
objc[2790]: Class CVView is implemented in both /Users/odner/opt/miniconda3/envs/torch/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x144d0e5b0) and /Users/odner/opt/miniconda3/envs/torch/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x116110a90). One of the two will be used. Which one is u

In [2]:
IMAGES_PATH = os.path.join('data', 'images')
EDIT_IMAGES_PATH = os.path.join('data', 'edit_images')
CORDS_EMOTION_PATH = os.path.join('data', 'cords_emotion.npy')
number_images = 30

arr_cords_emotion = np.array([["name", 'id', [('x', 'y', 'z')]]], dtype=object)
np.save(CORDS_EMOTION_PATH, arr_cords_emotion)

mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh
mp_drawing_styles = mp.solutions.drawing_styles

In [3]:
def dictMediapipeInArray(emotion_id, name, face_landmarks):
    global arr_cords_emotion
    
    arr_cords = list()
    for item in face_landmarks.landmark:
        arr_cords.append((item.x, item.y, item.z))
        
    with open(CORDS_EMOTION_PATH, 'rb') as file:
        arr = np.load(file, allow_pickle=True)
    with open(CORDS_EMOTION_PATH, 'wb') as file:
        arr = np.append(arr, [[name, emotion_id, arr_cords]], axis = 0)
        np.save(file, arr)

    del arr_cords

In [12]:
emotion_id = 1

In [15]:
cap = cv2.VideoCapture(0)
while True:
    ret, image = cap.read()
    
    with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True) as face_mesh:
        image.flags.writeable = False
        # Меняем цветовую гамму
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Либа по посторойке координат на лице
        results = face_mesh.process(image)
        image.flags.writeable = True
        # Возвращаем цвет обратно
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        if results.multi_face_landmarks:
            face_landmarks = results.multi_face_landmarks[0]
            mp_drawing.draw_landmarks(image=image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
               
    cv2.imshow('image', image)

    if cv2.waitKey(5) & 0xFF == 27: # exit on ESC
        break

cap.release()
cv2.destroyAllWindows()

cap = cv2.VideoCapture(0)
for imgnum in range(number_images):
    print(f'Image: {imgnum}')
    idimg = str(uuid.uuid4())
            
    ret, image = cap.read()
    
    imgname = os.path.join(IMAGES_PATH, f'{idimg}.jpg')
    cv2.imwrite(imgname, image)
            
    with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True) as face_mesh:
        image.flags.writeable = False
        # Меняем цветовую гамму
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Либа по посторойке координат на лице
        results = face_mesh.process(image)
        image.flags.writeable = True
        # Возвращаем цвет обратно
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
    
        if results.multi_face_landmarks:
            face_landmarks = results.multi_face_landmarks[0]
            mp_drawing.draw_landmarks(image=image,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                .get_default_face_mesh_tesselation_style())

            dictMediapipeInArray(idimg, emotion_id, face_landmarks)

    imgname = os.path.join(EDIT_IMAGES_PATH, f'{idimg}.jpg')
    cv2.imwrite(imgname, image)
    time.sleep(0.5)

    cv2.imshow('image', image)

    if cv2.waitKey(5) & 0xFF == 27: # exit on ESC
        break

cap.release()
cv2.destroyAllWindows()

Image: 0
Image: 1
Image: 2
Image: 3
Image: 4
Image: 5
Image: 6
Image: 7
Image: 8
Image: 9
Image: 10
Image: 11
Image: 12
Image: 13
Image: 14
Image: 15
Image: 16
Image: 17
Image: 18
Image: 19
Image: 20
Image: 21
Image: 22
Image: 23
Image: 24
Image: 25
Image: 26
Image: 27
Image: 28
Image: 29
